In [1]:
# Edge AI Image Classifier: Recyclable vs Non-Recyclable
# Built for Google Colab

# Step 1: Install Dependencies
!pip install tensorflow tensorflow-datasets

# Step 2: Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Step 3: Load & Prepare Dataset (we'll simulate with 'rock_paper_scissors' for binary classification demo)
(ds_train, ds_test), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

# Filter to only 2 classes: 'rock' vs 'paper' (simulate binary recyclable vs non-recyclable)
def filter_binary(image, label):
    return tf.logical_or(tf.equal(label, 0), tf.equal(label, 1))

def map_binary(image, label):
    label = tf.where(tf.equal(label, 0), 0, 1)  # rock=0, paper=1
    return image, label

ds_train = ds_train.filter(filter_binary).map(map_binary).batch(32).prefetch(1)
ds_test = ds_test.filter(filter_binary).map(map_binary).batch(32).prefetch(1)

# Step 4: Define Model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(300, 300, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Step 5: Train Model
model.fit(ds_train, validation_data=ds_test, epochs=5)

# Step 6: Evaluate
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.2f}")

# Step 7: Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Step 8: Save the .tflite model
with open('recycle_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite model saved!")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/incomplete.F35ZQ7_3.0.0/rock_paper_scissors-train.tfre…

...

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


     53/Unknown 125s 2s/step - accuracy: 0.5344 - loss: 2.4346

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


53/53 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5362 - loss: 2.4123 - val_accuracy: 0.6452 - val_loss: 0.5172
Epoch 2/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.9345 - loss: 0.2028 - val_accuracy: 0.9234 - val_loss: 0.1693
Epoch 3/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9934 - loss: 0.0292 - val_accuracy: 0.9274 - val_loss: 0.1709
Epoch 4/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9990 - loss: 0.0116 - val_accuracy: 0.9274 - val_loss: 0.1893
Epoch 5/5
53/53 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.9476 - val_loss: 0.1554
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 619ms/step - accuracy: 0.9466 - loss: 0.1682
Test Accuracy: 0.95
Saved artifact at '/tmp/tmpzvlzww4t'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1